In [44]:
import pandas as pd
import mysql.connector

In [22]:
# Path to CSV file
csv_path = r"C:\Users\boris\Documents\DataEngenier\Spark\final_project\input\SUB-IP-EST2023-CUMCHG.xlsx"

# Read Excel file with both header rows
df_raw = pd.read_excel(csv_path, sheet_name="SUB-IP-EST2023-CUMCHG", header=None) 

df_raw.head(5)

,0,1,2,3,4,5
0,table with row headers in column B and column ...,NaN,NaN,NaN,NaN,NaN
1,Cumulative Estimates of Resident Population Ch...,NaN,NaN,NaN,NaN,NaN
2,Rank,Geographic Area,Population Estimate,NaN,"Change, 2020 to 2023",NaN
3,NaN,NaN,2020-04-01 00:00:00,2023-07-01 00:00:00,Number,Percent
4,1,"Forney city, Texas",23463,35470,12007,51.2


In [24]:
# Extract the required columns
df_extracted = df_raw.iloc[4:, [1, 3]]  # Selecting "Geographic Area" and "July 1, 2023" Population Estimate

# Rename columns for clarity
df_extracted.columns = ["Geographic Area", "Population Estimate (July 1, 2023)"]

# Reset index
df_extracted = df_extracted.reset_index(drop=True)

# Display the cleaned data
df_extracted.head(5)

,Geographic Area,"Population Estimate (July 1, 2023)"
0,"Forney city, Texas",35470
1,"Georgetown city, Texas",96312
2,"Hutto city, Texas",38765
3,"Saratoga Springs city, Utah",52532
4,"Lathrop city, California",39857


In [25]:
# Split "Geographic Area" into city and state
df_extracted[["City", "State"]] = df_extracted["Geographic Area"].str.replace(" city", "", regex=True).str.split(", ", expand=True)

# Drop the original "Geographic Area" column
df_extracted = df_extracted.drop(columns=["Geographic Area"])

# Reorder columns
df_extracted = df_extracted[["City", "State", "Population Estimate (July 1, 2023)"]]

# Display the updated data
df_extracted.head(5)


,City,State,"Population Estimate (July 1, 2023)"
0,Forney,Texas,35470
1,Georgetown,Texas,96312
2,Hutto,Texas,38765
3,Saratoga Springs,Utah,52532
4,Lathrop,California,39857


In [26]:
df_extracted = df_extracted.rename(columns={"Population Estimate (July 1, 2023)": "population_Q2_2023"})
# Display the updated DataFrame
print(df_extracted.head(5))

               City       State population_Q2_2023
0            Forney       Texas              35470
1        Georgetown       Texas              96312
2             Hutto       Texas              38765
3  Saratoga Springs        Utah              52532
4           Lathrop  California              39857


In [40]:
df_WA = df_extracted[df_extracted['State'] == 'Washington']
display(df_WA.head())
df_WA.info()

,City,State,population_Q2_2023
68,Lynnwood,Washington,43867
125,Redmond,Washington,80280
178,Battle Ground,Washington,22285
277,Richland,Washington,63757
284,Spokane Valley,Washington,108235


<class 'pandas.core.frame.DataFrame'>
Index: 55 entries, 68 to 1694
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   City                55 non-null     object
 1   State               55 non-null     object
 2   population_Q2_2023  55 non-null     object
dtypes: object(3)
memory usage: 1.7+ KB


In [42]:
df_WA = df_WA.drop(columns=["State"])
df_WA.head()

,City,population_Q2_2023
68,Lynnwood,43867
125,Redmond,80280
178,Battle Ground,22285
277,Richland,63757
284,Spokane Valley,108235


In [47]:
import mysql.connector

# Connect to MySQL
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="mysql",  # replace with your MySQL password
    database="EV_Population_Data"
)
cursor = conn.cursor()

# Create the table 'washington_population' if it doesn't exist
create_table_query = """
CREATE TABLE IF NOT EXISTS washington_population (
    city VARCHAR(50) UNIQUE NOT NULL,
    population_Q2_2023 INTEGER NOT NULL
)
"""
cursor.execute(create_table_query)

# Convert DataFrame to list of tuples (city, population)
df_WA_list = [(row.City, row.population_Q2_2023) for row in df_WA.itertuples(index=False)]

# Insert each city and population into the table (using IGNORE to prevent duplicates)
insert_query = "INSERT IGNORE INTO washington_population (city, population_Q2_2023) VALUES (%s, %s)"
cursor.executemany(insert_query, df_WA_list)  # More efficient than a loop

# Commit the changes to the database
conn.commit()

print("Inserted unique city and population values into 'washington_population' table.")

# Close connection
cursor.close()
conn.close()


Inserted unique city and population values into 'washington_population' table.


In [48]:
import mysql.connector

# Connect to MySQL
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="mysql",  
    database="EV_Population_Data"  
)
cursor = conn.cursor()

# Define the SQL query
query = """
SELECT 
    a.city, 
    a.electric_cars, 
    w.population_Q2_2023, 
    a.electric_cars / NULLIF(w.population_Q2_2023, 0) AS ratio
FROM agg_cities a 
JOIN washington_population w 
ON a.city = w.city
"""

# Execute the query
cursor.execute(query)

# Fetch all results
results = cursor.fetchall()

# Print results
print("City | Electric Cars | Population | Ratio")
for row in results:
    print(row)

# Close the connection
cursor.close()
conn.close()


City | Electric Cars | Population | Ratio
('Auburn', 1632, 83870, Decimal('0.0195'))
('Bainbridge Island', 1566, 24254, Decimal('0.0646'))
('Battle Ground', 479, 22285, Decimal('0.0215'))
('Bellevue', 7691, 151574, Decimal('0.0507'))
('Bellingham', 2531, 94720, Decimal('0.0267'))
('Bonney Lake', 815, 22835, Decimal('0.0357'))
('Bothell', 4861, 50213, Decimal('0.0968'))
('Bremerton', 1028, 45450, Decimal('0.0226'))
('Burien', 736, 50730, Decimal('0.0145'))
('Camas', 1347, 27254, Decimal('0.0494'))
('Covington', 474, 21125, Decimal('0.0224'))
('Des Moines', 425, 31988, Decimal('0.0133'))
('Edmonds', 1590, 42701, Decimal('0.0372'))
('Everett', 2043, 111180, Decimal('0.0184'))
('Federal Way', 1207, 97701, Decimal('0.0124'))
('Issaquah', 2365, 38977, Decimal('0.0607'))
('Kenmore', 880, 23391, Decimal('0.0376'))
('Kennewick', 733, 85158, Decimal('0.0086'))
('Kent', 2276, 133378, Decimal('0.0171'))
('Kirkland', 4622, 91194, Decimal('0.0507'))
('Lacey', 778, 58326, Decimal('0.0133'))
('Lake St